In [ ]:
from langchain_core.tools import tool
from langchain_teddynote.tools import GoogleNews
from typing import List, Dict

from email.header import Header
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
import sys, os

In [ ]:

sys.path.append(os.path.abspath(os.path.join(os.path.dirname('get_naver_news'), '..')))
from module.get_mcp_client import get_mcp_client
from module.get_news_origin import get_news_origin
from module.get_local_pdf import rag_pdf



In [ ]:
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("CH15-Agents")

In [ ]:

# 도구 생성
@tool
def search_google_news(query: str) -> List[Dict[str, str]]:
    """Search Google News by input keyword"""
    news_tool = GoogleNews()
    return news_tool.search_by_keyword(query, k=5)


@tool
async def search_naver_news(query: str,k: int =10) -> List[Dict[str, str]]:
    """Search naver News by input keyword"""
    client = get_mcp_client()
    tools = await client.get_tools(server_name="naver-search-mcp")
    tool = next(tool for tool in tools if tool.name == "search_news")

    response = await tool.ainvoke({
        "query": query,
        "display": k,  # 최대 100
        "start": 1,
        "sort": "date"  # 날짜순 정렬
    })
    return get_news_origin(response)

@tool
def search_local_data(query: str) -> List[Dict[str, str]]:
    """Search local data of pdf file infomation by input keyword"""
    news_tool = rag_pdf()
    return news_tool.search_by_keyword(query)

from langchain.tools.retriever import create_retriever_tool

# search_local_data = create_retriever_tool(
#     retriever= ,
#     name = "pdf_search",
#     description="use this tool to search information from the PDF document"
# )

@tool
async def send_google_mail(email:str, body: str,subject:str = "안녕하세요") -> List[Dict[str, str]]:
    """A function that can send an email by receiving an address and subject and body"""
    client = get_mcp_client()
    tools = await client.get_tools(server_name="gmail-mcp")
    tool = next(tool for tool in tools if tool.name == "send_message")
    encoded_subject = Header(subject, 'utf-8').encode()
    response = await tool.ainvoke({
        "to":[email],
        "subject":encoded_subject,
        "body":body,
    })
    return response

# tools = [search_google_news,search_local_data]


tools = [search_google_news, search_naver_news,search_local_data,send_google_mail]


In [ ]:
from langchain_core.prompts import ChatPromptTemplate

# 프롬프트 생성
# 프롬프트는 에이전트에게 모델이 수행할 작업을 설명하는 텍스트를 제공합니다. (도구의 이름과 역할을 입력)
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. "
            # "Make sure to use the `search_google_news` tool for searching keyword related news in google site. Make sure to use the `search_naver_news` tool for searching keyword related news in naver site.",

        ),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
)

In [ ]:
from langchain_openai import ChatOpenAI
from langchain.agents import create_tool_calling_agent

llm = ChatOpenAI(model="gpt-4o-mini",temperature =0 )
agent = create_tool_calling_agent(llm, tools, prompt)

In [ ]:
from langchain.agents import AgentExecutor

# AgentExecutor 생성
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
    max_iterations=10,
    max_execution_time=30,
    handle_parsing_errors=True,
)

In [ ]:
from langchain_teddynote.messages import AgentStreamParser
agent_stream_parser = AgentStreamParser()


In [ ]:
query = "사랑하는 사람에게 올해 잘해보자는 따뜻한 메시지를 담아 ansgyqja@naver.com 에게 전송해줘"

# result =await agent_executor.ainvoke({"input":query})
# print(result["output"])

async for step in agent_executor.astream({"input":query}):
    agent_stream_parser.process_agent_steps(step)

In [ ]:
# session_id 를 저장할 딕셔너리 생성
store = {}


# session_id 를 기반으로 세션 기록을 가져오는 함수
def get_session_history(session_ids):
    if session_ids not in store:  # session_id 가 store에 없는 경우
        # 새로운 ChatMessageHistory 객체를 생성하여 store에 저장
        store[session_ids] = ChatMessageHistory()
    return store[session_ids]  # 해당 세션 ID에 대한 세션 기록 반환


# 채팅 메시지 기록이 추가된 에이전트를 생성합니다.
agent_with_chat_history = RunnableWithMessageHistory(
    agent_executor,
    # 대화 session_id
    get_session_history,
    # 프롬프트의 질문이 입력되는 key: "input"
    input_messages_key="input",
    # 프롬프트의 메시지가 입력되는 key: "chat_history"
    history_messages_key="chat_history",
)


In [ ]:
# 질의에 대한 답변을 스트리밍으로 출력 요청
query = "안녕 ?"
async for step in agent_with_chat_history.astream(
    {"input": query},
    # session_id 설정
    config={"configurable": {"session_id": "abc123"}},
):
    agent_stream_parser.process_agent_steps(step)



In [ ]:
# 질의에 대한 답변을 스트리밍으로 출력 요청
query = "메일 보내고싶은데"
async for step in agent_with_chat_history.astream(
    {"input": query},
    # session_id 설정
    config={"configurable": {"session_id": "abc123"}},
):
    agent_stream_parser.process_agent_steps(step)



In [ ]:
# 질의에 대한 답변을 스트리밍으로 출력 요청
query = "ansgyqja@naver.com 에게 오늘 날씨 정보와 생일 축하 메시지를 전달해줘"
async for step in agent_with_chat_history.astream(
    {"input": query},
    # session_id 설정
    config={"configurable": {"session_id": "abc123"}},
):
    agent_stream_parser.process_agent_steps(step)



In [ ]:
# 질의에 대한 답변을 스트리밍으로 출력 요청
query = "제목은 사자성어로 된 올해가 잘풀렷으면 좋겟다는 의미르 가졌으면 좋겟고, 내용은 올해 운수 대박나라는 귀엽고 깜직한 html 양식으로 보내줘  "
async for step in agent_with_chat_history.astream(
    {"input": query},
    # session_id 설정
    config={"configurable": {"session_id": "abc123"}},
):
    agent_stream_parser.process_agent_steps(step)

